In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys

print("installed finish")

installed finish


In [9]:
from selenium.webdriver.support.ui import Select
option = Options()
option.add_argument("--disable-notifications")

chrome = webdriver.Chrome('./chromedriver', chrome_options=option)

url = "http://www.shanghairanking.com/rankings/gras/2021"
chrome.get(url)
soup = BeautifulSoup(chrome.page_source, 'html.parser')

C:\Users\1234\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [15]:
## get 51 subject name
url = "http://www.shanghairanking.com/rankings/gras/2021"
response = requests.get(url)
soup = BeautifulSoup(response.text, "lxml")
# print(soup)

title_list = []

for title in titles:
    title_list.append(title.text)
    print(title.text)

Natural Sciences
Engineering

Life Sciences
Medical Sciences
Social Sciences



In [19]:
subject_dict = {}
subjects = soup.find_all(class_="subj-link")

for subject in subjects:
    link = subject["href"].split("/")[-1]
    subject_dict[subject.text] = link
    print(link, subject.text)

RS0101 Mathematics
RS0102 Physics
RS0103 Chemistry
RS0104 Earth Sciences
RS0105 Geography
RS0106 Ecology
RS0107 Oceanography
RS0108 Atmospheric Science
RS0201 Mechanical Engineering
RS0202 Electrical & Electronic Engineering
RS0205 Automation & Control
RS0206 Telecommunication Engineering
RS0207 Instruments Science & Technology
RS0208 Biomedical Engineering
RS0210 Computer Science & Engineering
RS0211 Civil Engineering
RS0212 Chemical Engineering
RS0213 Materials Science & Engineering
RS0214 Nanoscience & Nanotechnology
RS0215 Energy Science & Engineering
RS0216 Environmental Science & Engineering
RS0217 Water Resources
RS0219 Food Science & Technology
RS0220 Biotechnology
RS0221 Aerospace Engineering
RS0222 Marine/Ocean Engineering
RS0223 Transportation Science & Technology
RS0224 Remote Sensing
RS0226 Mining & Mineral Engineering
RS0227 Metallurgical Engineering
RS0301 Biological Sciences
RS0302 Human Biological Sciences
RS0303 Agricultural Sciences
RS0304 Veterinary Sciences
RS0401 

In [153]:
# find total page number
def get_final_page_number(sub_num):
    subject_url = "http://www.shanghairanking.com/rankings/gras/2021/" + sub_num

    chrome = webdriver.Chrome('./chromedriver', chrome_options=option)
    chrome.get(subject_url)

    soup = BeautifulSoup(chrome.page_source, 'lxml')
    flag = False
    tags_a = soup.find_all("a")
    limited_page = 0
    for tag in tags_a:
        if(tag.text == "•••" and flag == False):
            flag = True
        elif(flag == True):
            limited_page = int(tag.text)
            break
    print("limit:", limited_page)
    chrome.close()
    return limited_page

In [176]:
subject_url = "http://www.shanghairanking.com/rankings/gras/2021/RS0103"

chrome = webdriver.Chrome('./chromedriver', chrome_options=option)
chrome.get(subject_url)

for i in range(2):
    time.sleep(5)
    for index in range(5):
            # 點擊指標按鈕
            btns = chrome.find_elements_by_class_name("head-bg", )
            print(len(btns))
            btns[-1].click()

            links = chrome.find_elements_by_tag_name("li")
            link_list = []
            # for 5 score
            btn_name_list = ["Q1", "CNCI", "IC", "TOP", "AWARD"]
            for link in links:
                if(link.text == btn_name_list[index]):
                    # 選擇目前指標
                    link.click() 
                    # extract html content
                    soup = BeautifulSoup(chrome.page_source, 'lxml')
                    get_page_info(soup, btn_name_list[index])
                    print(btn_name_list[index])
                    time.sleep(2)
                    break
    # click next page
    btns_next_page = chrome.find_elements_by_class_name("ant-pagination-item-link", )
    btns_next_page[-1].click()
    print(len(btns_next_page))
    # 滾軸拉到最上面 (才可以點擊分項按鈕)
    js="var action=document.documentElement.scrollTop=0"
    chrome.execute_script(js)
    time.sleep(1)
    

C:\Users\1234\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


2
Q1
2
CNCI
2
IC
2
TOP
2
AWARD
3
2
Q1
2
CNCI
2
IC
2
TOP
2
AWARD
3


In [197]:
def list_append_all(result, list1, list2, list3, list4):
    list1 += result[0]
    list2 += result[1]
    list3[0] += result[2]
    list4 += result[3]

In [200]:
# click next button
subject_url = "http://www.shanghairanking.com/rankings/gras/2021/RS0101"

chrome = webdriver.Chrome('./chromedriver', chrome_options=option)
chrome.get(subject_url)

count = 0
limited_page = get_final_page_number("RS0101")
univ_name = []
total_score = []
item_score = [[] for i in range(5)]
ranking = []
while(count < limited_page):
    time.sleep(1)
    print("Page:", count + 1)
    # 5 score for a page
    for index in range(5):
        btns = chrome.find_elements_by_class_name("head-bg", )
        btns[-1].click()

        links = chrome.find_elements_by_tag_name("li")
        # for 5 score
        btn_name_list = ["Q1", "CNCI", "IC", "TOP", "AWARD"]
        for link in links:
            if(link.text == btn_name_list[index]):
                link.click()
                soup = BeautifulSoup(chrome.page_source, 'lxml')
                result = get_page_info(soup, btn_name_list[index])
                if(index == 0):
                    list_append_all(result, ranking, total_score, item_score, univ_name)
                else:
                    item_score[index] += result[2]
                time.sleep(2)
                break
    # click next page
    btns_next_page = chrome.find_elements_by_class_name("ant-pagination-item-link", )
    btns_next_page[-1].click()
    count += 1
    js="var action=document.documentElement.scrollTop=0"
    chrome.execute_script(js)
    time.sleep(1)
    print()
    
chrome.close()
print(count)

C:\Users\1234\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.
C:\Users\1234\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


limit: 17
Page: 1
This is Q1 Page.
This is CNCI Page.
This is IC Page.
This is TOP Page.
This is AWARD Page.

Page: 2
This is Q1 Page.
['31', '168.2', '66']  row error.
This is CNCI Page.
['31', '168.2', '72']  row error.
This is IC Page.
['31', '168.2', '74.6']  row error.
This is TOP Page.
['31', '168.2', '51.3']  row error.
This is AWARD Page.
['31', '168.2', '0']  row error.

Page: 3
This is Q1 Page.
This is CNCI Page.
This is IC Page.
This is TOP Page.
This is AWARD Page.

Page: 4
This is Q1 Page.
This is CNCI Page.
This is IC Page.
This is TOP Page.
This is AWARD Page.

Page: 5
This is Q1 Page.
This is CNCI Page.
This is IC Page.
This is TOP Page.
This is AWARD Page.

Page: 6
This is Q1 Page.
This is CNCI Page.
This is IC Page.
This is TOP Page.
This is AWARD Page.

Page: 7
This is Q1 Page.
This is CNCI Page.
This is IC Page.
This is TOP Page.
This is AWARD Page.

Page: 8
This is Q1 Page.
This is CNCI Page.
This is IC Page.
This is TOP Page.
This is AWARD Page.

Page: 9
This is Q

In [201]:
print(ranking, total_score, item_score, univ_name)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '27', '29', '30', '32', '33', '34', '35', '36', '37', '38', '39', '39', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '51-75', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '76-100', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150', '101-150

In [202]:
import pandas as pd
import numpy as np

In [205]:
df = pd.DataFrame()
df["Year"] = ["2021" for i in range(len(ranking))]
df["Field"] = ["Natural Sciences" for i in range(len(ranking))]
df["Subject"] = ["Mathematics" for i in range(len(ranking))]
df["World Rank"] = ranking
df["Institution"] = univ_name
df["Total Score"] = total_score
name_list = ["Q1", "CNCI", "IC", "TOP", "AWARD"]
for index in range(len(name_list)):
    df[name_list[index]] = item_score[index]
# df.head()
df.to_csv("ARWU-subject/Mathematics.csv", index=False)
print("finish")

In [191]:
# get 5 score page
subject_url = "http://www.shanghairanking.com/rankings/gras/2021/RS0101"

chrome = webdriver.Chrome('./chromedriver', chrome_options=option)
chrome.get(subject_url)

index = 0
for i in range(5):
    btns = chrome.find_elements_by_class_name("head-bg", )
    btns[-1].click()

    links = chrome.find_elements_by_tag_name("li")
    link_list = []
    # for 5 score
    btn_name_list = ["Q1", "CNCI", "IC", "TOP", "AWARD"]
    for link in links:
        if(link.text == btn_name_list[index]):
            link.click()
            soup = BeautifulSoup(chrome.page_source, 'lxml')
            get_page_info(soup, link.text)
            time.sleep(2)
            break
    index += 1

C:\Users\1234\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


This is  Page.
This is  Page.
This is  Page.
This is  Page.
This is  Page.


In [186]:
def get_page_info(soup, tag):
    print("This is", tag, "Page.")
    items = soup.find_all("tr")
    flag = False
    lists = [[], [], [], []]

    for row in items:
        if(flag == False):
            flag = True
            continue
        single_list = []
        # score and ranking
        for i in [0,3,4]:
            try:
                text = row.contents[i].text.lstrip().rstrip()
                single_list.append(text)
            except:
                pass
        # university name
        try:
            text = row.find(class_="univ-name").text.lstrip().rstrip()
            single_list.append(text)
        except:
            pass
        
        if(len(single_list) == 4):
            for i in range(4):
                lists[i].append(single_list[i])
        else:
            print(single_list, " row error.")
#     print(lists)
    return lists

This is Q1 Page.
[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '27', '29', '30'], ['328.5', '311.2', '269.1', '262.9', '256.2', '251.3', '243.5', '243.1', '234.1', '227.2', '219.7', '218.4', '214.4', '212.8', '209.3', '200.6', '199.5', '198.1', '197.6', '196.5', '195.6', '189.1', '187.6', '181.2', '178.9', '176.6', '175.5', '175.5', '169.8', '169.3'], ['71', '88.1', '70.4', '88.7', '73.2', '86.6', '83.6', '93.2', '76.7', '82.8', '82.3', '77.2', '79.5', '76.6', '77.5', '80.6', '81.3', '69.7', '76.5', '78.2', '70', '68.7', '66.5', '60.7', '83.3', '64.5', '64.6', '68', '73.3', '86.2'], ['Paris-Saclay University', 'Princeton University', 'Sorbonne University', 'University of Cambridge', 'University of Oxford', 'Stanford University', 'Massachusetts Institute of Technology (MIT)', 'New York University', 'ETH Zurich', 'The University of Texas at Austin', 'University of California, Los 

In [24]:
subject_url = "http://www.shanghairanking.com/rankings/gras/2021/RS0101"

chrome = webdriver.Chrome('./chromedriver', chrome_options=option)
chrome.get(subject_url)

btns = chrome.find_elements_by_class_name("head-bg", )
btns[-1].click()

soup = BeautifulSoup(chrome.page_source, 'lxml')

C:\Users\1234\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until
